# Project 23-24
###### By Stefano Morgese 861431 - Mattia Deiana 844668

# Librerie usate

In [1]:
#IMPORTO LIBRERIE UTILI
import pandas as pd
import sqlite3
import requests

# Funzioni utili

* per dataset

In [2]:
#funzione caricamento dataset corse dei bus
def data1(nd):
    #caricamento file metaData.csv (Viaggi dei Bus) da GitHub
    url = "https://raw.githubusercontent.com/MatD99/project_FCS/main/project_file/" + nd
    d = pd.read_csv(url, parse_dates=['startTime_iso','endTime_iso'])
    #formattare data startTime_iso (più leggibile)    
    d['startTime_iso'] = pd.to_datetime(d['startTime_iso'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    #formattare data endTime_iso (più leggibile)
    d['endTime_iso'] = pd.to_datetime(d['endTime_iso'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    
    #restituzione del dataset
    return(d)

In [3]:
#funzione creazione dataset due dalla cartella ZTbus
def data2(df):    
    #percorso per recuperare nome file nella cartella ZTbus_samples per confrontare esistenza con file metaData
    urlC = "https://api.github.com/repos/MatD99/project_FCS/contents/project_file/ZTbus_samples"
    #urlC = "https://api.github.com/repos/MatD99/project_FCS/contents/project_file/ZTbus"
    #usare token per avere poter effettuare la richiesta soprastante senza problemi
    token = ""
    headers = {'Authorization': f'token {token}'}
    response = requests.get(urlC, headers=headers)
    #lista per nomi file
    files = []   
    #verificare se richiesta andata a buon fine, se si allora salvo nome file 
    if response.status_code == 200:
        data = response.json()
        for item in data:
            if item['type'] == 'file':
                files.append(item['name'])  
    
    #variabile booleana che controlla se primo dataset è stato concatenato
    first = False
    
    #confrontare i nomi di files, con quelli presenti nel dataset1 (verifico se Trips, ha dataset associato in ZTbus_samples) 
    for file in files:
        for i in range(len(df)):
            name = df.loc[i, 'name']
            if (name + '.csv') == file:               
                #selezionare il dataset in ZTBus_samples                
                url2 = "https://raw.githubusercontent.com/MatD99/project_FCS/main/project_file/ZTbus_samples/" + file
                #url2 = "https://raw.githubusercontent.com/MatD99/project_FCS/main/project_file/ZTbus/"
                dfx = pd.read_csv(url2, parse_dates=['time_iso'])
                #creazione chiave esterna in dataset2, utile per join
                dfx['name'] = name                
                #concatenazione dataset
                if first == False:
                    #primo dataset aggiunto da solo
                    dfy = dfx
                    #dopo aver trovato il primo dataset setto la variabile a true (in quanto primo file concatenato)
                    first = True
                else:
                    #non essendo più il primo dataset, concatenazione dei vari dataset al dataset2
                    dfy = pd.concat([dfy, dfx])
                    
    #resettare indice per una selezione corretta futura dei record
    dfy = dfy.reset_index()
    #cancellazione colonna index
    dfy = dfy.drop('index', axis=1)
    #formattare data time_iso (più leggibile)
    dfy['time_iso'] = pd.to_datetime(dfy['time_iso'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    
    #restituzione dataset2
    return(dfy)

* per query

In [4]:
#funzione esecuzione query su un dataset
def single_query(query, df, ds):
    #connessione al database SQLite (crea un database in memoria)
    conn = sqlite3.connect(':memory:')

    #salvataggio DataFrame pandas nel database SQLite come tabella
    df.to_sql(ds, conn, index=False)

    #esecuzione query desiderata sul database    
    res = pd.read_sql_query(query, conn)

    #chiusura connessione database
    conn.close()
    
    #restituzione risultato
    return(res)

In [5]:
#funzione esecuzione query utilizzando dataset1 e dataset2 (join tabelle)
def inner_query(query, df, ds):
    #connessione al database SQLite
    conn = sqlite3.connect('database.db')
    
    #salvataggio join DataFrame in SQLite    
    df.to_sql('busDataTrips', conn, if_exists='replace', index=False)
    ds.to_sql('busDataTripsCharacteristics', conn, if_exists='replace', index=False)
    
    #esecuzione query desiderata sul database
    res = pd.read_sql_query(query, conn)
    
    #chiusura connessione database
    conn.close()
    
    #restituzione risultato
    return(res)

# Dataset

In [6]:
#DATASET 1 metaData.csv
df1 = data1("metaData.csv") 
#df1.head(2)
df1

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56,1556594336,2019-04-30 08:44:20,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30 13:22:07,1556630527,2019-04-30 17:54:02,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01 05:58:51,1556690331,2019-05-01 22:32:30,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21,1556851821,2019-05-03 05:53:20,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03 15:41:57,1556898117,2019-05-03 23:06:24,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06 14:43:49,1670337829,2022-12-06 18:22:52,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02,1670389982,2022-12-07 19:19:53,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08 05:22:20,1670476940,2022-12-08 18:39:15,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12,1670630112,2022-12-10 03:24:28,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500


In [7]:
#DATASET 2 ottenuto da cartella ZTbus_samples
#creazione dataset2
df2 = data2(data1("metaData.csv")) 
#df2.head(2)
df2

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_rl,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,name
0,2019-06-24 03:16:13,1561346173,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06
1,2019-06-24 03:16:14,1561346174,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06
2,2019-06-24 03:16:15,1561346175,0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06
3,2019-06-24 03:16:16,1561346176,-0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06
4,2019-06-24 03:16:17,1561346177,-0.00000,393.2,2.17119,0.826972,0.148489,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485495,2022-11-30 19:50:18,1669837818,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22
485496,2022-11-30 19:50:19,1669837819,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22
485497,2022-11-30 19:50:20,1669837820,18.27846,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22
485498,2022-11-30 19:50:21,1669837821,18.27631,NaN,NaN,NaN,NaN,-,NaN,-,...,0.0,0.0,1.0,0.0,0.0,1.0,281.15,0.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22


# Query

### 1. Extract all trips with busRoute 83

In [8]:
#Q1_python
df1[df1['busRoute'] == '83']

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
154,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03 04:42:38,1583210558,2020-03-03 19:44:51,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
155,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06 04:53:23,1583470403,2020-03-06 19:44:42,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
157,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09 14:16:13,1583763373,2020-03-09 19:34:17,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
158,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10 04:50:03,1583815803,2020-03-10 19:51:25,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
159,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12 04:56:41,1583989001,2020-03-12 19:44:57,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1399,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30 04:47:53,1669783673,2022-11-30 19:50:22,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
1400,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01 05:19:41,1669871981,2022-12-01 18:20:57,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
1401,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02 04:47:48,1669956468,2022-12-02 19:40:01,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02,1670389982,2022-12-07 19:19:53,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


In [9]:
#Q1_SQL
query_1 = single_query("SELECT * FROM busDataTrips WHERE busRoute = 83", df1, 'busDataTrips')
#risultati della query 1
query_1

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2020-03-03_04-42-38_2020-03-03_19-44-51,183,2020-03-03 04:42:38,1583210558,2020-03-03 19:44:51,1583264691,225047.90,83,1.544278e+09,23.47531,0,118,0.472180,280.5450,279.15,289.1500
1,B183_2020-03-06_04-53-23_2020-03-06_19-44-42,183,2020-03-06 04:53:23,1583470403,2020-03-06 19:44:42,1583523882,224512.30,83,1.631816e+09,17.41578,0,69,0.451028,279.8850,278.15,289.1500
2,B183_2020-03-09_14-16-13_2020-03-09_19-34-17,183,2020-03-09 14:16:13,1583763373,2020-03-09 19:34:17,1583782457,77824.36,83,5.406013e+08,23.18182,0,74,0.460099,281.0489,279.15,291.1500
3,B183_2020-03-10_04-50-03_2020-03-10_19-51-25,183,2020-03-10 04:50:03,1583815803,2020-03-10 19:51:25,1583869885,225095.80,83,1.692171e+09,20.96410,0,86,0.475233,279.8363,279.15,291.1500
4,B183_2020-03-12_04-56-41_2020-03-12_19-44-57,183,2020-03-12 04:56:41,1583989001,2020-03-12 19:44:57,1584042297,224181.20,83,1.145860e+09,17.21235,0,80,0.340882,287.3445,282.15,291.1500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
841,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,208,2022-11-30 04:47:53,1669783673,2022-11-30 19:50:22,1669837822,223165.00,83,1.560888e+09,27.89066,2,100,0.456196,280.6948,279.15,293.1500
842,B208_2022-12-01_05-19-41_2022-12-01_18-20-57,208,2022-12-01 05:19:41,1669871981,2022-12-01 18:20:57,1669918857,190196.00,83,1.418847e+09,26.03927,0,96,0.450413,279.7655,279.15,292.1500
843,B208_2022-12-02_04-47-48_2022-12-02_19-40-01,208,2022-12-02 04:47:48,1669956468,2022-12-02 19:40:01,1670010001,224473.40,83,1.611150e+09,24.80384,2,91,0.438693,279.7888,279.15,291.1500
844,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02,1670389982,2022-12-07 19:19:53,1670440793,210041.60,83,1.536697e+09,28.78539,0,115,0.434858,279.5283,278.15,292.6655


### 2. Extract all trips where busRoute is not a number 

In [10]:
#Q2_python
df1[~df1['busRoute'].str.isnumeric()]

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56,1556594336,2019-04-30 08:44:20,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21,1556851821,2019-05-03 05:53:20,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
9,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10 03:16:11,1557458171,2019-05-10 18:51:37,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
10,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13 03:10:23,1557717023,2019-05-13 23:16:13,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
19,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24 02:52:47,1558666367,2019-05-24 22:35:11,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1373,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21 22:38:32,1666391912,2022-10-22 02:42:21,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
1374,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22 22:34:45,1666478085,2022-10-23 02:29:59,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
1394,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25 23:35:16,1669419316,2022-11-26 03:30:39,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12,1670630112,2022-12-10 03:24:28,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


In [11]:
#Q2_SQL 
query_2 = single_query("SELECT * FROM busDataTrips WHERE busRoute GLOB '*[^0-9]*'", df1, 'busDataTrips')
#risultati della query 2
query_2

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56,1556594336,2019-04-30 08:44:20,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.15
1,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21,1556851821,2019-05-03 05:53:20,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.15
2,B183_2019-05-10_03-16-11_2019-05-10_18-51-37,183,2019-05-10 03:16:11,1557458171,2019-05-10 18:51:37,1557514297,210577.00,-,1.303391e+09,8.230483,0,43,0.740927,287.5623,282.15,293.15
3,B183_2019-05-13_03-10-23_2019-05-13_23-16-13,183,2019-05-13 03:10:23,1557717023,2019-05-13 23:16:13,1557789373,267033.80,-,1.647432e+09,7.891652,0,45,0.804191,284.6764,280.15,293.15
4,B183_2019-05-24_02-52-47_2019-05-24_22-35-11,183,2019-05-24 02:52:47,1558666367,2019-05-24 22:35:11,1558737311,263432.60,-,1.448057e+09,7.520249,0,44,0.761068,293.1440,283.15,299.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,B208_2022-10-21_22-38-32_2022-10-22_02-42-21,208,2022-10-21 22:38:32,1666391912,2022-10-22 02:42:21,1666406541,78567.16,N1,4.347766e+08,16.333330,0,45,0.431852,289.2550,288.15,296.15
80,B208_2022-10-22_22-34-45_2022-10-23_02-29-59,208,2022-10-22 22:34:45,1666478085,2022-10-23 02:29:59,1666492199,73427.97,N2,3.997737e+08,17.710530,0,57,0.443358,287.3486,285.15,295.15
81,B208_2022-11-25_23-35-16_2022-11-26_03-30-39,208,2022-11-25 23:35:16,1669419316,2022-11-26 03:30:39,1669433439,72911.26,N2,4.475534e+08,11.216670,1,32,0.465024,281.3884,280.15,293.15
82,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12,1670630112,2022-12-10 03:24:28,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.15


### 3. For each (busNumber, busRoute) pair, determine the number of trips

In [12]:
#Q3_python
#risultato finale non considera 'busRoute' con valore "-", poichè inteso come null
trip_counts = df1[df1['busRoute'] != "-"]
trip_counts = trip_counts.groupby(['busNumber', 'busRoute']).size().reset_index(name='numberOfTrips')#trip_counts = trip_counts.groupby('busRoute')['busNumber'].value_counts().reset_index(name = 'numberOfTrips')
#risultato finale non considera 'busRoute' con il seguente valore "-", poichè inteso come valore null
trip_counts[['busNumber', 'busRoute', 'numberOfTrips']]

,busNumber,busRoute,numberOfTrips
0,183,31,12
1,183,32,12
2,183,33,130
3,183,46,104
4,183,72,114
5,183,83,441
6,183,N1,10
7,183,N2,19
8,183,N4,11
9,208,31,5


In [13]:
#Q3_SQL
#risultato finale non considera 'busRoute' con valore "-", poichè inteso come null
query_3 = single_query("SELECT busNumber, busRoute, count(*) as numberOfTrips FROM busDataTrips WHERE busRoute <> '-' GROUP BY busNumber, busRoute ", df1, 'busDataTrips')
#risultati della query 3
query_3

,busNumber,busRoute,numberOfTrips
0,183,31,12
1,183,32,12
2,183,33,130
3,183,46,104
4,183,72,114
5,183,83,441
6,183,N1,10
7,183,N2,19
8,183,N4,11
9,208,31,5


### 4. For each trip, compute the ratio between the energy consumption and the average number of passengers

In [14]:
#caricamento dataset1
ratio = data1("metaData.csv")

In [15]:
#Q4_python
#ratio calcolo
ratio['energyConsumptionToPassengerRatio'] = ratio['energyConsumption'] / ratio['itcs_numberOfPassengers_mean']
ratio

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,energyConsumptionToPassengerRatio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56,1556594336,2019-04-30 08:44:20,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30 13:22:07,1556630527,2019-04-30 17:54:02,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01 05:58:51,1556690331,2019-05-01 22:32:30,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21,1556851821,2019-05-03 05:53:20,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03 15:41:57,1556898117,2019-05-03 23:06:24,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500,2.613190e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06 14:43:49,1670337829,2022-12-06 18:22:52,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500,1.070215e+07
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02,1670389982,2022-12-07 19:19:53,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655,5.338462e+07
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08 05:22:20,1670476940,2022-12-08 18:39:15,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500,4.738047e+07
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12,1670630112,2022-12-10 03:24:28,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500,2.247753e+07


In [16]:
#Q4_SQL
query_4 = single_query("SELECT *, (energyConsumption/itcs_numberOfPassengers_mean) as energyConsumptionToPassengerRatio FROM busDataTrips", df1, 'busDataTrips')
#risultati della query 4
query_4

,name,busNumber,startTime_iso,startTime_unix,endTime_iso,endTime_unix,drivenDistance,busRoute,energyConsumption,itcs_numberOfPassengers_mean,itcs_numberOfPassengers_min,itcs_numberOfPassengers_max,status_gridIsAvailable_mean,temperature_ambient_mean,temperature_ambient_min,temperature_ambient_max,energyConsumptionToPassengerRatio
0,B183_2019-04-30_03-18-56_2019-04-30_08-44-20,183,2019-04-30 03:18:56,1556594336,2019-04-30 08:44:20,1556613860,77213.87,-,4.785852e+08,5.538860,0,20,0.740640,282.3780,281.15,293.1500,8.640500e+07
1,B183_2019-04-30_13-22-07_2019-04-30_17-54-02,183,2019-04-30 13:22:07,1556630527,2019-04-30 17:54:02,1556646842,59029.60,31,4.022585e+08,33.114580,4,74,0.855234,287.5443,285.15,293.1500,1.214747e+07
2,B183_2019-05-01_05-58-51_2019-05-01_22-32-30,183,2019-05-01 05:58:51,1556690331,2019-05-01 22:32:30,1556749950,240900.40,33,1.445733e+09,19.689140,0,55,0.777860,288.7490,280.15,294.1500,7.342794e+07
3,B183_2019-05-03_02-50-21_2019-05-03_05-53-20,183,2019-05-03 02:50:21,1556851821,2019-05-03 05:53:20,1556862800,42565.48,-,2.819867e+08,1.685185,0,8,0.767122,282.4129,281.15,292.1500,1.673328e+08
4,B183_2019-05-03_15-41-57_2019-05-03_23-06-24,183,2019-05-03 15:41:57,1556898117,2019-05-03 23:06:24,1556924784,125277.20,72,6.207258e+08,23.753570,1,67,0.907342,284.7325,282.15,287.1500,2.613190e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,B208_2022-12-06_14-43-49_2022-12-06_18-22-52,208,2022-12-06 14:43:49,1670337829,2022-12-06 18:22:52,1670350972,51798.78,32,4.260419e+08,39.808990,0,83,0.739349,279.6404,278.15,291.1500,1.070215e+07
1405,B208_2022-12-07_05-13-02_2022-12-07_19-19-53,208,2022-12-07 05:13:02,1670389982,2022-12-07 19:19:53,1670440793,210041.60,83,1.536697e+09,28.785390,0,115,0.434858,279.5283,278.15,292.6655,5.338462e+07
1406,B208_2022-12-08_05-22-20_2022-12-08_18-39-15,208,2022-12-08 05:22:20,1670476940,2022-12-08 18:39:15,1670524755,190372.70,83,1.415700e+09,29.879400,0,102,0.439916,279.1724,277.15,292.1500,4.738047e+07
1407,B208_2022-12-09_23-55-12_2022-12-10_03-24-28,208,2022-12-09 23:55:12,1670630112,2022-12-10 03:24:28,1670642668,59548.57,N1,4.519165e+08,20.105260,0,74,0.495739,279.4540,277.15,291.1500,2.247753e+07


### 5. For each station (itcs_stopName), determine the average number of passengers.

In [17]:
#Q5_python
#risultato finale non considera 'itcs_stopName' con valore "-", poichè inteso come null
average_passengers_per_station = df2[df2['itcs_stopName'] != "-"]
average_passengers_per_station = average_passengers_per_station.groupby('itcs_stopName')['itcs_numberOfPassengers'].mean().reset_index()
average_passengers_per_station

,itcs_stopName,itcs_numberOfPassengers
0,"Zürich, Albisrank",17.791209
1,"Zürich, Albisriederplatz",26.946237
2,"Zürich, Altes Krematorium",22.198758
3,"Zürich, Bahnhof Altstetten",10.297872
4,"Zürich, Bahnhof Hardbrücke",28.251337
...,...,...
71,"Zürich, Vogelsangstrasse",15.897959
72,"Zürich, Waffenplatz-/Bederstr.",16.434783
73,"Zürich, Waffenplatzstrasse",20.729167
74,"Zürich, Wildbachstrasse",3.952381


In [18]:
#Q5_SQL
#risultato finale non considera 'itcs_stopName' con valore "-", poichè inteso come null
query_5 = single_query("SELECT itcs_stopName, AVG(itcs_numberOfPassengers) as avgOfPassengers FROM busDataTripsCharacteristics WHERE itcs_stopName <> '-' GROUP BY itcs_stopName", df2, 'busDataTripsCharacteristics')
#risultati della query 5 (arrotondati all'intero poichè persone)
query_5.round(0)

,itcs_stopName,avgOfPassengers
0,"Zürich, Albisrank",18.0
1,"Zürich, Albisriederplatz",27.0
2,"Zürich, Altes Krematorium",22.0
3,"Zürich, Bahnhof Altstetten",10.0
4,"Zürich, Bahnhof Hardbrücke",28.0
...,...,...
71,"Zürich, Vogelsangstrasse",16.0
72,"Zürich, Waffenplatz-/Bederstr.",16.0
73,"Zürich, Waffenplatzstrasse",21.0
74,"Zürich, Wildbachstrasse",4.0


### 6. For each station, determine the buses (busNumber) that have stopped there at least once.

In [19]:
#join tra i due dataset
buses_per_station = pd.merge(df1, df2, on = 'name')

In [20]:
#Q6_python 
#busNumber è considerato come BUS, risultato finale non considera 'itcs_stopName' con valore "-", poichè inteso come null
#busNumber che si sono fermati almeno una volta in quella stazione 
buses_per_station = buses_per_station[buses_per_station['itcs_stopName'] != '-']
buses_per_station = buses_per_station.groupby(['itcs_stopName','busNumber']).size().reset_index(name = 'numberOfStops')#buses_per_station = buses_per_station.groupby('itcs_stopName')['busNumber'].value_counts().reset_index(name = 'numberOfStops')
buses_per_station[['itcs_stopName','busNumber']]

,itcs_stopName,busNumber
0,"Zürich, Albisrank",183
1,"Zürich, Albisrank",208
2,"Zürich, Albisriederplatz",183
3,"Zürich, Albisriederplatz",208
4,"Zürich, Altes Krematorium",183
...,...,...
119,"Zürich, Waffenplatz-/Bederstr.",208
120,"Zürich, Waffenplatzstrasse",208
121,"Zürich, Wildbachstrasse",183
122,"Zürich, Wildbachstrasse",208


In [21]:
#Q6_SQL
#risultato finale non considera 'itcs_stopName' con valore "-", poichè inteso come null
query_6 = inner_query("SELECT itcs_stopName, busNumber, COUNT(busNumber) as numberOfStops FROM busDataTrips t1 INNER JOIN busDataTripsCharacteristics t2 ON t1.name = t2.name WHERE itcs_stopName <> '-' GROUP BY itcs_stopName, busNumber HAVING COUNT(busNumber) >= 1", df1, df2)
#risultati della query 6
query_6[['itcs_stopName','busNumber']]
#query_6

,itcs_stopName,busNumber
0,"Zürich, Albisrank",183
1,"Zürich, Albisrank",208
2,"Zürich, Albisriederplatz",183
3,"Zürich, Albisriederplatz",208
4,"Zürich, Altes Krematorium",183
...,...,...
119,"Zürich, Waffenplatz-/Bederstr.",208
120,"Zürich, Waffenplatzstrasse",208
121,"Zürich, Wildbachstrasse",183
122,"Zürich, Wildbachstrasse",208


### 7. For each station, determine the buses that have stopped there at least ten times.

In [22]:
#Q7_python (riutilizzo Q6_python)
buses_at_least_ten_stops = buses_per_station[buses_per_station['numberOfStops'] >= 10].reset_index()
buses_at_least_ten_stops[['itcs_stopName','busNumber']]

,itcs_stopName,busNumber
0,"Zürich, Albisrank",183
1,"Zürich, Albisrank",208
2,"Zürich, Albisriederplatz",183
3,"Zürich, Albisriederplatz",208
4,"Zürich, Altes Krematorium",183
...,...,...
95,"Zürich, Waffenplatz-/Bederstr.",208
96,"Zürich, Waffenplatzstrasse",208
97,"Zürich, Wildbachstrasse",183
98,"Zürich, Wildbachstrasse",208


In [23]:
#Q7_SQL
query_7 = inner_query("SELECT itcs_stopName, busNumber, COUNT(busNumber) as numberOfStops FROM busDataTrips t1 INNER JOIN busDataTripsCharacteristics t2 ON t1.name = t2.name WHERE itcs_stopName <> '-' GROUP BY itcs_stopName, busNumber HAVING COUNT(busNumber) >= 10", df1, df2)
#risultati della query 7
query_7[['itcs_stopName','busNumber']]
#query_7

,itcs_stopName,busNumber
0,"Zürich, Albisrank",183
1,"Zürich, Albisrank",208
2,"Zürich, Albisriederplatz",183
3,"Zürich, Albisriederplatz",208
4,"Zürich, Altes Krematorium",183
...,...,...
95,"Zürich, Waffenplatz-/Bederstr.",208
96,"Zürich, Waffenplatzstrasse",208
97,"Zürich, Wildbachstrasse",183
98,"Zürich, Wildbachstrasse",208


### 9. For each (route, bus) pair, compute the ratio between the overall energy consumption and the overall driven distance.

In [24]:
#Q9_python
#risultato finale non considera 'busRoute' con valore "-", poichè inteso come null
energy_distance_sum = df1[df1['busRoute'] != "-"]
energy_distance_sum = energy_distance_sum.groupby(['busRoute', 'busNumber']).agg({'energyConsumption': 'sum', 'drivenDistance': 'sum'}).reset_index()
energy_distance_sum['energyToDistanceRatio'] = energy_distance_sum['energyConsumption'] / energy_distance_sum['drivenDistance']
energy_distance_sum

,busRoute,busNumber,energyConsumption,drivenDistance,energyToDistanceRatio
0,31,183,1.387782e+10,2279952.00,6086.891785
1,31,208,5.535865e+09,1046306.68,5290.862713
2,32,183,1.393320e+10,2256540.10,6174.585331
3,32,208,1.283340e+10,2337064.21,5491.247671
4,33,183,1.761996e+11,29510481.41,5970.747507
5,33,208,3.474842e+10,6161892.39,5639.244537
6,46,183,1.026237e+11,18224657.94,5631.035959
7,46,208,2.337366e+10,4186493.70,5583.110707
8,72,183,1.394368e+11,23637238.45,5899.029643
9,72,208,5.100382e+10,9427138.15,5410.318878


In [25]:
#Q9_SQL
#risultato finale non considera 'busRoute' con valore "-", poichè inteso come null
query_9 = single_query("SELECT busNumber, busRoute, sum(energyConsumption) AS energyConsumption, sum(drivenDistance) AS drivenDistance, (sum(energyConsumption)/sum(drivenDistance)) as energyToDistanceRatio FROM busDataTrips WHERE busRoute <> '-' GROUP BY busNumber, busRoute", df1, 'busDataTrips')
#risultati della query 9
query_9

,busNumber,busRoute,energyConsumption,drivenDistance,energyToDistanceRatio
0,183,31,1.387782e+10,2279952.00,6086.891785
1,183,32,1.393320e+10,2256540.10,6174.585331
2,183,33,1.761996e+11,29510481.41,5970.747507
3,183,46,1.026237e+11,18224657.94,5631.035959
4,183,72,1.394368e+11,23637238.45,5899.029643
5,183,83,2.995540e+11,51249813.70,5844.977154
6,183,N1,4.217283e+09,705122.71,5980.920966
7,183,N2,7.929098e+09,1390618.35,5701.850332
8,183,N4,4.720068e+09,766868.89,6154.986937
9,208,31,5.535865e+09,1046306.68,5290.862713


### 10. Starting from the results of the previous point, for each route compute the buses with max and min energy ratio, and save the difference between these ratios in a dataframe.

In [26]:
#Q10_python (riutilizzo Q9_python)
#trovare gli indici riferiti alla cella del dataframe che contengono valore più alto e più basso (groupby('busRoute'))
max_ratio_indices = energy_distance_sum.groupby('busRoute')['energyToDistanceRatio'].idxmax()
min_ratio_indices = energy_distance_sum.groupby('busRoute')['energyToDistanceRatio'].idxmin()

#associore gli indici ai valori
max_ratio_buses = energy_distance_sum.loc[max_ratio_indices]
min_ratio_buses = energy_distance_sum.loc[min_ratio_indices]

#differenza tra le misure e inserimento in un nuovo dataframe
difference = max_ratio_buses.set_index('busRoute')['energyToDistanceRatio'] - min_ratio_buses.set_index('busRoute')['energyToDistanceRatio']
difference_df = pd.DataFrame({'busRoute': difference.index, 'difference_ratios': difference.values})
difference_df

,busRoute,difference_ratios
0,31,796.029072
1,32,683.337660
2,33,331.502969
3,46,47.925252
4,72,488.710765
5,83,25.338263
6,N1,340.859083
7,N2,296.697369
8,N4,87.796825


In [27]:
#Q10_SQL (riutilizzo Q9_SQL)
query_10 = single_query("SELECT busRoute, (MAX(energyToDistanceRatio)-MIN(energyToDistanceRatio)) AS difference_ratios FROM busDataTrips GROUP BY busRoute", query_9, 'busDataTrips')
#risultati della query 10
query_10

,busRoute,difference_ratios
0,31,796.029072
1,32,683.337660
2,33,331.502969
3,46,47.925252
4,72,488.710765
5,83,25.338263
6,N1,340.859083
7,N2,296.697369
8,N4,87.796825


### 11. Find the bus maximizing the difference computed in the previous point.

In [28]:
#Q11_python (riutilizzo Q10_python)
# #tramite indice del valore max
# max_difference_bus = difference_df['difference_ratios'].idxmax()
# max_difference_value = difference_df.loc[max_difference_bus]
# print("Bus maximizing the difference:")
# print(max_difference_value)
#con valore max
max_difference_bus = difference_df['difference_ratios'].max()
max_difference_value = difference_df.loc[difference_df['difference_ratios'] == max_difference_bus]
max_difference_value

,busRoute,difference_ratios
0,31,796.029072


In [29]:
#Q11_SQL (riutilizzo Q10_SQL)
query_11 = single_query("SELECT busRoute, MAX(difference_ratios) AS maximizingDifference FROM busDataTrips", query_10, 'busDataTrips')
#risultati della query 11
query_11

,busRoute,maximizingDifference
0,31,796.029072


### 12. Extract the rows of the details such that the gnss_altitude differs from the value in the preceding row. Store also the difference in the variable altitude_variation.

In [30]:
#creazione dataset2 da ZTbus_samples per Q12
df3 = data2(data1('metaData.csv')) 

In [31]:
#Q12_python
#creazione colonna altitude_variation per inserimento dei valori
df3['altitude_variation'] = df3['gnss_altitude'].diff().apply(lambda x: x if pd.notnull(x) and x != 0 else 0)
#selezionare record in cui si verifica una un risultato diverso da 0
altitude_variation = df3[(df3['altitude_variation'] != 0)].reset_index()
altitude_variation = altitude_variation.drop(columns=['index'])
altitude_variation

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,name,altitude_variation
0,2019-06-24 03:16:39,1561346199,0.000,393.1023,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,-0.0977
1,2019-06-24 03:16:40,1561346200,-0.000,393.1976,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0953
2,2019-06-24 03:16:41,1561346201,-0.000,393.2000,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0024
3,2019-06-24 03:16:45,1561346205,-0.000,393.2976,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0976
4,2019-06-24 03:16:46,1561346206,-0.000,393.3000,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425166,2022-11-30 19:49:29,1669837769,7718.101,434.6953,3.377910,0.826910,0.148451,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,242500.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,0.1065
425167,2022-11-30 19:49:30,1669837770,7821.313,436.4188,3.383577,0.826904,0.148449,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,242500.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,1.7235
425168,2022-11-30 19:49:31,1669837771,7915.435,436.9681,3.369137,0.826900,0.148448,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,241666.7,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,0.5493
425169,2022-11-30 19:49:32,1669837772,2728.455,438.3276,3.349103,0.826896,0.148448,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,241666.7,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,1.3595


In [32]:
#Q12_SQL
query_12 = single_query("SELECT * FROM busDataTripsCharacteristics WHERE altitude_variation <> 0" , df3, 'busDataTripsCharacteristics')
#risultati della query 12
query_12

,time_iso,time_unix,electric_powerDemand,gnss_altitude,gnss_course,gnss_latitude,gnss_longitude,itcs_busRoute,itcs_numberOfPassengers,itcs_stopName,...,odometry_wheelSpeed_rr,status_doorIsOpen,status_gridIsAvailable,status_haltBrakeIsActive,status_parkBrakeIsActive,temperature_ambient,traction_brakePressure,traction_tractionForce,name,altitude_variation
0,2019-06-24 03:16:39,1561346199,0.000,393.1023,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,-0.0977
1,2019-06-24 03:16:40,1561346200,0.000,393.1976,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0953
2,2019-06-24 03:16:41,1561346201,0.000,393.2000,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0024
3,2019-06-24 03:16:45,1561346205,0.000,393.2976,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0976
4,2019-06-24 03:16:46,1561346206,0.000,393.3000,2.171190,0.826972,0.148489,-,NaN,-,...,0.0,0.0,0.0,0.0,1.0,296.15,246666.7,0.0,B183_2019-06-24_03-16-13_2019-06-24_18-54-06,0.0024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425166,2022-11-30 19:49:29,1669837769,7718.101,434.6953,3.377910,0.826910,0.148451,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,242500.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,0.1065
425167,2022-11-30 19:49:30,1669837770,7821.313,436.4188,3.383577,0.826904,0.148449,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,242500.0,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,1.7235
425168,2022-11-30 19:49:31,1669837771,7915.435,436.9681,3.369137,0.826900,0.148448,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,241666.7,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,0.5493
425169,2022-11-30 19:49:32,1669837772,2728.455,438.3276,3.349103,0.826896,0.148448,-,NaN,-,...,0.0,1.0,0.0,1.0,1.0,282.15,241666.7,0.0,B208_2022-11-30_04-47-53_2022-11-30_19-50-22,1.3595


### 13. For each details dataset, compute the sum of the absolute value (i.e. the sign is not considered) of altitude_variation.

In [33]:
#Q13_python (riutilizzo Q10_python)
abs_altitude_variation = altitude_variation['altitude_variation'].abs().sum()
abs_altitude_variation

126782.23040000001

In [34]:
#Q13_SQL (riutilizzo Q10_SQL)
query_13 = single_query("SELECT SUM(ABS(altitude_variation)) AS absAltitudeVariation FROM busDataTripsCharacteristics" , query_12, 'busDataTripsCharacteristics')
#risultati della query 13
query_13

,absAltitudeVariation
0,126782.2304


### 14. For each month of the year, compute the average ambient temperature

In [35]:
#creazione dataset2 da ZTbus_samples per Q15
df4 = data2(data1('metaData.csv'))

In [36]:
#Q14_python
df4['month'] = pd.to_datetime(df4['time_iso']).dt.strftime('%Y-%m')
average_ambient_temp_per_month = df4.groupby('month')['temperature_ambient'].mean().reset_index()
average_ambient_temp_per_month[['month', 'temperature_ambient']].sort_values(by='temperature_ambient', ascending=True)

,month,temperature_ambient
8,2022-11,280.660145
2,2021-05,287.819472
5,2021-10,289.159768
7,2022-10,289.700618
3,2021-07,292.460708
4,2021-09,295.668297
1,2020-07,297.107453
0,2019-06,299.968397
6,2022-07,301.345118


In [37]:
#Q14_SQL
query_14 = single_query("SELECT strftime('%Y-%m', time_iso) AS month, AVG(temperature_ambient) AS avgAmbient_temperature FROM busDataTripsCharacteristics GROUP BY month ORDER BY temperature_ambient", df4, 'busDataTripsCharacteristics')
#risultati della query 14
query_14

,month,avgAmbient_temperature
0,2022-11,280.660145
1,2021-10,289.159768
2,2020-07,297.107453
3,2021-05,287.819472
4,2021-07,292.460708
5,2021-09,295.668297
6,2022-10,289.700618
7,2019-06,299.968397
8,2022-07,301.345118


### 15. For each bus compute the total time when the halt brake is active and the total time when the park brake is active. Compute also the ratio between those two times.

In [38]:
#creazione dataset2 da ZTbus_samples per Q15
df5 = data2(data1('metaData.csv'))

In [39]:
#contare tempo quando è attivo status_haltBrakeIsActive
df5['timeUnix_haltBrakeIsActive'] = df5['status_haltBrakeIsActive'].shift().apply(lambda x: 1 if x == 1.0 else 0)
#contare tempo quando è attivo status_parkBrakeIsActive
df5['timeUnix_parkBrakeIsActive'] = df5['status_parkBrakeIsActive'].apply(lambda x: 1 if x == 1.0 else 0)

In [40]:
#Q15_python
#join dei due dataset per trovare risultato finale
dfjoin = pd.merge(df5, df1, on = 'name')
query_15 = dfjoin.groupby('busNumber')[['timeUnix_haltBrakeIsActive', 'timeUnix_parkBrakeIsActive']].sum().reset_index()
#formattare tempo da time_unix in ora minuti e secondi
query_15['time_haltBrakeIsActive'] = (pd.to_datetime(query_15['timeUnix_haltBrakeIsActive'], unit='s')).dt.strftime('%H:%M:%S')
query_15['time_parkBrakeIsActive'] = (pd.to_datetime(query_15['timeUnix_parkBrakeIsActive'], unit='s')).dt.strftime('%H:%M:%S')
query_15[['busNumber','time_haltBrakeIsActive','time_parkBrakeIsActive']]

,busNumber,time_haltBrakeIsActive,time_parkBrakeIsActive
0,183,08:17:07,04:58:59
1,208,06:15:52,10:15:54


In [41]:
#Q15_SQL
query_15 = inner_query("SELECT busNumber, sum(timeUnix_haltBrakeIsActive) AS timeUnix_haltBrakeIsActive, sum(timeUnix_parkBrakeIsActive) AS timeUnix_parkBrakeIsActive FROM busDataTrips t1 INNER JOIN busDataTripsCharacteristics t2 ON t1.name = t2.name GROUP BY busnumber", df1, df5)
#formattare tempo da time_unix in ora minuti e second
query_15['time_haltBrakeIsActive'] = (pd.to_datetime(query_15['timeUnix_haltBrakeIsActive'], unit='s')).dt.strftime('%H:%M:%S')
query_15['time_parkBrakeIsActive'] = (pd.to_datetime(query_15['timeUnix_parkBrakeIsActive'], unit='s')).dt.strftime('%H:%M:%S')
query_15[['busNumber','time_haltBrakeIsActive','time_parkBrakeIsActive']]

,busNumber,time_haltBrakeIsActive,time_parkBrakeIsActive
0,183,08:17:07,04:58:59
1,208,06:15:52,10:15:54


In [42]:
#CALCOLO RATIO E VISUALIZZAZIONE FINALE query_15
#creazione nuova colonna
query_15['ratio_brake'] = 0.0
#ratio per time_haltBrakeIsActive
query_15.loc[0, 'ratio_brake'] = query_15['timeUnix_haltBrakeIsActive'].iloc[0] / query_15['timeUnix_parkBrakeIsActive'].iloc[0] 
#ratio per time_parkBrakeIsActive
query_15.loc[1, 'ratio_brake'] = query_15['timeUnix_haltBrakeIsActive'].iloc[1] / query_15['timeUnix_parkBrakeIsActive'].iloc[1]
#visualizzazione risultato finale della query 15
query_15 = query_15[['busNumber','time_haltBrakeIsActive','time_parkBrakeIsActive', 'ratio_brake']]
query_15

,busNumber,time_haltBrakeIsActive,time_parkBrakeIsActive,ratio_brake
0,183,08:17:07,04:58:59,1.662690
1,208,06:15:52,10:15:54,2.948314


# 16. For each pair of stops that are consecutive in at least a trip, compute the average speed achieved when going from the first to the second stop.

In [43]:
#creazione dataset2 da ZTbus_samples per Q16
df6 = data2(data1('metaData.csv'))

In [44]:
#liste dove vengono inseriti i dati utili per la query finale
stop = []
next_stop = []
dayTrip = []
nextTrip = []
stopC = []
cont = 0

#fermate già salvate in ordine di tempo
#salvataggio delle fermate
df6['itcs_stopName'].apply(lambda x: stop.append(x) if x !="-" else None)

#ciclo che mi premette di salvare l'indice per capire quando inizia e finisce una femata e se effettuata nello stesso giorno
for c in range(len(df6)):
    if("-" != df6['itcs_stopName'].iloc[c] ):
        stopC.append(c)       
        dayTrip.append(df6['time_iso'].iloc[c])

#salvataggio fermata successiva
next_stop = stop[1:]
#fermate di partenza
stop = stop[:-1]
#indice partenza prima fermata (assoluta)
s = stopC[0]
#indice partenza fermate successive
stopC = stopC[1:]
#giorno e tempo fermata
nextTrip = dayTrip[1:]
dayTrip = dayTrip[:-1]

In [45]:
#alcune variabili utili
load = False # pullman non è partito
tot = 0 #indice utile per salvare i vari dati
#liste finali che verranno inserite nel dataframe finale
stop2 = [] #tutta la tratta delle fermate di partenza
next_stop2 = [] #tutte le tratte delle fermate successive
dayTrip2 = [] #tutto l'elenco dei giorni delle fermate di partenza 
nextTrip2 = [] #tutto l'elenco dei giorni delle fermate successive 

#ciclo che permette di salvare tutte le info sulle varie tratte (fermata, fermata successiva, girono fermata)
for c in range(len(df6)):
    if (c == s) | (load != False):
        load = True

        if (tot >= len(stopC) - 1):
            stop2.append(None)
            next_stop2.append(None)
            dayTrip2.append(None)
            nextTrip2.append(None)
        else:
            if (c> stopC[tot]):
                tot +=1                       
            stop2.append(stop[tot])
            next_stop2.append(next_stop[tot])
            dayTrip2.append(dayTrip[tot])
            nextTrip2.append(nextTrip[tot]) 
    else:
        stop2.append(None)
        next_stop2.append(None)
        dayTrip2.append(None)
        nextTrip2.append(None)

In [46]:
#salvataggio dati nel dataframe
df6['dayTrip'] = df6.apply(lambda row, dayTrip2 = dayTrip2: dayTrip2[row.name], axis = 1)
df6['dayTrip2'] = df6.apply(lambda row, dayTrip2 = dayTrip2: dayTrip2[row.name], axis = 1)
df6['startStop'] = df6.apply(lambda row, stop2 = stop2: stop2[row.name], axis = 1)
df6['nextStop'] = df6.apply(lambda row, next_stop2 = next_stop2: next_stop2[row.name], axis = 1)

In [47]:
#selezione le fermate esistenti della tratta
pairOfTrips = df6[df6['startStop'].notna()]
#verificare se le fermate consecutive sono state effettuate lo stesso giorno
pairOfTrips = pairOfTrips[pairOfTrips['dayTrip'] == pairOfTrips['dayTrip2']]
#calcolare velocità media
pairOfTrips = pairOfTrips.groupby(['startStop', 'nextStop']).agg({'dayTrip2': 'first', 'odometry_vehicleSpeed': 'mean'}).sort_values(by='dayTrip2', ascending=True)
pairOfTrips = pairOfTrips.rename(columns={'dayTrip2': 'timeStop'})
pairOfTrips.reset_index()

,startStop,nextStop,timeStop,odometry_vehicleSpeed
0,"Zürich, Albisriederplatz","Zürich, Hardplatz",2019-06-24 03:25:54,4.505983
1,"Zürich, Hardplatz","Zürich, Bahnhof Hardbrücke",2019-06-24 03:29:00,5.105686
2,"Zürich, Bahnhof Hardbrücke","Zürich, Schiffbau",2019-06-24 03:30:29,6.437221
3,"Zürich, Schiffbau","Zürich, Escher-Wyss-Platz",2019-06-24 03:31:14,5.825235
4,"Zürich, Escher-Wyss-Platz","Zürich, Rosengartenstrasse",2019-06-24 03:31:59,6.003615
...,...,...,...,...
133,"Zürich, Bertastrasse","Zürich, Friedhof Sihlfeld",2022-07-16 06:57:07,5.280326
134,"Zürich, Friedhof Sihlfeld","Zürich, Altes Krematorium",2022-07-16 06:57:56,5.728480
135,"Zürich, Besenrainstrasse","Zürich, Albisriederplatz",2022-07-16 17:49:25,2.954338
136,"Zürich, Besenrainstrasse","Zürich, Milchbuck",2022-10-26 15:09:58,2.588838


In [48]:
#Q16_SQL
query_16 = single_query(
'''SELECT startStop, nextStop, dayTrip2 AS stopTime, AVG(odometry_vehicleSpeed) AS avgSpeed
   FROM (   
      SELECT *
      FROM busDataTripsCharacteristics
      WHERE startStop IS NOT NULL AND dayTrip = dayTrip2      
   ) AS subquery
   GROUP BY startStop, nextStop
   ORDER BY stopTime''',
df6, 'busDataTripsCharacteristics')
#risultati della query 16 
query_16

,startStop,nextStop,stopTime,avgSpeed
0,"Zürich, Albisriederplatz","Zürich, Hardplatz",2019-06-24 03:25:54,4.505983
1,"Zürich, Hardplatz","Zürich, Bahnhof Hardbrücke",2019-06-24 03:29:00,5.105686
2,"Zürich, Bahnhof Hardbrücke","Zürich, Schiffbau",2019-06-24 03:30:29,6.437221
3,"Zürich, Schiffbau","Zürich, Escher-Wyss-Platz",2019-06-24 03:31:14,5.825235
4,"Zürich, Escher-Wyss-Platz","Zürich, Rosengartenstrasse",2019-06-24 03:31:59,6.003615
...,...,...,...,...
133,"Zürich, Bertastrasse","Zürich, Friedhof Sihlfeld",2022-07-16 06:57:07,5.280326
134,"Zürich, Friedhof Sihlfeld","Zürich, Altes Krematorium",2022-07-16 06:57:56,5.728480
135,"Zürich, Besenrainstrasse","Zürich, Albisriederplatz",2022-07-16 17:49:25,2.954338
136,"Zürich, Besenrainstrasse","Zürich, Milchbuck",2022-10-26 15:09:58,2.588838
